<a href="https://colab.research.google.com/github/nmansour67/skills-introduction-to-github/blob/main/ER_Acuity_Score_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
"""
Emergency Room Acuity Score Calculator for Seasonal Flu Patients
Based on Emergency Severity Index (ESI) and vital signs assessment
Designed for Google Colab deployment

Author: Healthcare Analytics Consultant
Purpose: Educational resource for healthcare professionals learning Python
"""

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

class ERFlugAcuityScorer:
    """
    Emergency Room Acuity Scorer for Seasonal Flu Patients

    Acuity Levels (Based on ESI - Emergency Severity Index):
    Level 1 (Score: 1): Immediate life-threatening condition
    Level 2 (Score: 2): High risk, severe symptoms requiring immediate intervention
    Level 3 (Score: 3): Moderate risk, stable but requires multiple resources
    Level 4 (Score: 4): Low risk, stable, requires one resource
    Level 5 (Score: 5): Minimal risk, may not require resources

    For Seasonal Flu Assessment, we consider:
    - Vital Signs (Temperature, HR, RR, BP, SpO2)
    - Age (elderly and very young are higher risk)
    - Clinical presentation severity
    """

    def __init__(self):
        self.acuity_descriptions = {
            1: "Critical - Immediate Life-Threatening",
            2: "Emergent - High Risk, Immediate Intervention",
            3: "Urgent - Moderate Risk, Multiple Resources",
            4: "Less Urgent - Low Risk, One Resource",
            5: "Non-Urgent - Minimal Risk"
        }

    def generate_patient_data(self, n_patients=20):
        """
        Generate synthetic patient data for seasonal flu cases

        Parameters:
        n_patients (int): Number of patients to generate

        Returns:
        pd.DataFrame: Patient data with MRN, demographics, and vital signs
        """

        patients = []

        for i in range(n_patients):
            # Generate patient demographics
            mrn = f"MRN{str(2024120001 + i)}"
            age = random.choice([
                random.randint(2, 12),    # Children
                random.randint(18, 35),   # Young adults
                random.randint(36, 60),   # Middle-aged
                random.randint(65, 85)    # Elderly
            ])

            # Generate vital signs with some variability
            # Normal ranges with flu-related variations

            # Temperature (°F): Normal 97.0-99.0, Flu can cause 100.0-104.0
            temp = round(random.uniform(99.0, 103.5), 1)

            # Heart Rate (bpm): Normal 60-100, can be elevated with fever
            if age < 12:
                hr = random.randint(80, 140)  # Children have higher baseline
            elif age >= 65:
                hr = random.randint(70, 110)
            else:
                hr = random.randint(75, 120)

            # Respiratory Rate (breaths/min): Normal 12-20
            rr = random.randint(14, 28)

            # Systolic Blood Pressure (mmHg)
            if age >= 65:
                sbp = random.randint(110, 160)
            else:
                sbp = random.randint(100, 140)

            # Diastolic Blood Pressure (mmHg)
            if age >= 65:
                dbp = random.randint(65, 95)
            else:
                dbp = random.randint(60, 90)

            # Oxygen Saturation (SpO2): Normal >95%, can drop with respiratory complications
            spo2 = random.randint(88, 99)

            patients.append({
                'MRN': mrn,
                'Age': age,
                'Temperature_F': temp,
                'Heart_Rate_bpm': hr,
                'Respiratory_Rate': rr,
                'Systolic_BP': sbp,
                'Diastolic_BP': dbp,
                'SpO2_percent': spo2
            })

        return pd.DataFrame(patients)

    def calculate_acuity_score(self, row):
        """
        Calculate acuity score based on vital signs and age

        Scoring Logic:
        - Each vital sign abnormality contributes points
        - Age extremes (very young or elderly) add risk
        - Final score maps to ESI Level (1-5)

        Parameters:
        row (pd.Series): Patient data row

        Returns:
        tuple: (acuity_score, formula_explanation)
        """

        points = 0
        formula_parts = []

        # Temperature Assessment (Higher weight for severe fever)
        if row['Temperature_F'] >= 103.0:
            points += 3
            formula_parts.append("Temp≥103°F(+3)")
        elif row['Temperature_F'] >= 101.5:
            points += 2
            formula_parts.append("Temp≥101.5°F(+2)")
        elif row['Temperature_F'] >= 100.4:
            points += 1
            formula_parts.append("Temp≥100.4°F(+1)")
        else:
            formula_parts.append("Temp<100.4°F(+0)")

        # Heart Rate Assessment (Tachycardia concern)
        if row['Age'] < 12:
            threshold_high = 130
        elif row['Age'] >= 65:
            threshold_high = 100
        else:
            threshold_high = 110

        if row['Heart_Rate_bpm'] > threshold_high:
            points += 2
            formula_parts.append(f"HR>{threshold_high}(+2)")
        elif row['Heart_Rate_bpm'] > (threshold_high - 10):
            points += 1
            formula_parts.append(f"HR>{threshold_high-10}(+1)")
        else:
            formula_parts.append(f"HR≤{threshold_high-10}(+0)")

        # Respiratory Rate Assessment
        if row['Respiratory_Rate'] > 24:
            points += 2
            formula_parts.append("RR>24(+2)")
        elif row['Respiratory_Rate'] > 20:
            points += 1
            formula_parts.append("RR>20(+1)")
        else:
            formula_parts.append("RR≤20(+0)")

        # Blood Pressure Assessment (Hypotension concern)
        if row['Systolic_BP'] < 90:
            points += 3
            formula_parts.append("SBP<90(+3)")
        elif row['Systolic_BP'] < 100:
            points += 1
            formula_parts.append("SBP<100(+1)")
        else:
            formula_parts.append("SBP≥100(+0)")

        # Oxygen Saturation Assessment (Critical parameter)
        if row['SpO2_percent'] < 90:
            points += 4
            formula_parts.append("SpO2<90%(+4)")
        elif row['SpO2_percent'] < 93:
            points += 2
            formula_parts.append("SpO2<93%(+2)")
        elif row['SpO2_percent'] < 95:
            points += 1
            formula_parts.append("SpO2<95%(+1)")
        else:
            formula_parts.append("SpO2≥95%(+0)")

        # Age Risk Factor
        if row['Age'] < 5 or row['Age'] >= 65:
            points += 2
            if row['Age'] < 5:
                formula_parts.append("Age<5(+2)")
            else:
                formula_parts.append("Age≥65(+2)")
        elif row['Age'] < 12:
            points += 1
            formula_parts.append("Age<12(+1)")
        else:
            formula_parts.append("Age:18-64(+0)")

        # Map total points to ESI Acuity Level
        # Lower score = Higher acuity (more critical)
        if points >= 10:
            acuity_level = 2  # Emergent
        elif points >= 7:
            acuity_level = 3  # Urgent
        elif points >= 4:
            acuity_level = 4  # Less Urgent
        else:
            acuity_level = 5  # Non-Urgent

        # Build formula string
        formula_string = " + ".join(formula_parts) + f" = {points} points → ESI Level {acuity_level}"

        return acuity_level, formula_string, points

    def process_patients(self, df):
        """
        Process all patients and calculate acuity scores

        Parameters:
        df (pd.DataFrame): Patient dataframe

        Returns:
        pd.DataFrame: Enhanced dataframe with acuity scores and formulas
        """

        acuity_scores = []
        formulas = []
        total_points = []
        descriptions = []

        for idx, row in df.iterrows():
            score, formula, points = self.calculate_acuity_score(row)
            acuity_scores.append(score)
            formulas.append(formula)
            total_points.append(points)
            descriptions.append(self.acuity_descriptions[score])

        df['Total_Points'] = total_points
        df['Acuity_Score_ESI'] = acuity_scores
        df['Acuity_Description'] = descriptions
        df['Calculation_Formula'] = formulas

        return df


# ============================================================================
# MAIN EXECUTION
# ============================================================================

def main():
    """
    Main execution function for ER Acuity Scoring
    """

    print("="*80)
    print("EMERGENCY ROOM ACUITY SCORE CALCULATOR")
    print("Seasonal Flu Patients - December 2024")
    print("="*80)
    print()

    # Initialize the scorer
    scorer = ERFlugAcuityScorer()

    # Generate patient data
    print("Generating synthetic patient data for 20 ER admissions...")
    patients_df = scorer.generate_patient_data(n_patients=20)

    # Calculate acuity scores
    print("Calculating acuity scores based on vital signs and age...")
    results_df = scorer.process_patients(patients_df)

    # Sort by acuity score (most critical first)
    results_df = results_df.sort_values('Acuity_Score_ESI').reset_index(drop=True)

    print("Processing complete!")
    print()

    # Display results in organized format
    print("="*80)
    print("RESULTS SUMMARY")
    print("="*80)
    print()

    # Display full results table
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    # Create display dataframe with selected columns in logical order
    display_df = results_df[[
        'MRN',
        'Age',
        'Temperature_F',
        'Heart_Rate_bpm',
        'Respiratory_Rate',
        'Systolic_BP',
        'Diastolic_BP',
        'SpO2_percent',
        'Total_Points',
        'Acuity_Score_ESI',
        'Acuity_Description',
        'Calculation_Formula'
    ]].copy()

    # Rename columns for better readability
    display_df.columns = [
        'MRN',
        'Age',
        'Temp(°F)',
        'HR(bpm)',
        'RR(/min)',
        'SBP(mmHg)',
        'DBP(mmHg)',
        'SpO2(%)',
        'Points',
        'ESI',
        'Acuity Level',
        'Scoring Formula'
    ]

    print(display_df.to_string(index=False))
    print()

    # Display statistical summary
    print("="*80)
    print("ACUITY DISTRIBUTION SUMMARY")
    print("="*80)
    print()

    acuity_counts = results_df['Acuity_Score_ESI'].value_counts().sort_index()

    for level in sorted(acuity_counts.index):
        count = acuity_counts[level]
        percentage = (count / len(results_df)) * 100
        description = scorer.acuity_descriptions[level]
        print(f"ESI Level {level} ({description}): {count} patients ({percentage:.1f}%)")

    print()
    print("="*80)
    print("CLINICAL INTERPRETATION GUIDE")
    print("="*80)
    print()
    print("ESI Level 1: Requires immediate physician evaluation and intervention")
    print("ESI Level 2: Should be seen within 10 minutes")
    print("ESI Level 3: Multiple diagnostic or therapeutic interventions needed")
    print("ESI Level 4: One diagnostic or therapeutic intervention needed")
    print("ESI Level 5: No resources needed except provider examination")
    print()

    return results_df

# Execute the analysis
if __name__ == "__main__":
    results_dataframe = main()

    # Optional: Export to CSV for further analysis
    # results_dataframe.to_csv('er_acuity_scores_december_2024.csv', index=False)
    # print("Results exported to CSV file.")

EMERGENCY ROOM ACUITY SCORE CALCULATOR
Seasonal Flu Patients - December 2024

Generating synthetic patient data for 20 ER admissions...
Calculating acuity scores based on vital signs and age...
Processing complete!

RESULTS SUMMARY

          MRN  Age  Temp(°F)  HR(bpm)  RR(/min)  SBP(mmHg)  DBP(mmHg)  SpO2(%)  Points  ESI                               Acuity Level                                                                                                 Scoring Formula
MRN2024120003   72     101.7       70        26        120         87       94       7    3 Urgent - Moderate Risk, Multiple Resources     Temp≥101.5°F(+2) + HR≤90(+0) + RR>24(+2) + SBP≥100(+0) + SpO2<95%(+1) + Age≥65(+2) = 7 points → ESI Level 3
MRN2024120012    5     102.4      135        15        109         80       90       7    3 Urgent - Moderate Risk, Multiple Resources    Temp≥101.5°F(+2) + HR>130(+2) + RR≤20(+0) + SBP≥100(+0) + SpO2<93%(+2) + Age<12(+1) = 7 points → ESI Level 3
MRN2024120013   67     100